# Medical Dataset

Los conjuntos de datos de infarto se recopilaron en el hospital Zheen de Erbil (Irak) entre enero y mayo de 2019. Los atributos de este conjunto de datos son: edad, sexo, frecuencia cardíaca, presión arterial sistólica, presión arterial diastólica, glucemia, CK-MB y troponina con resultados negativos o positivos.

Según la información proporcionada, el conjunto de datos médicos clasifica si se trata de un infarto o no. La columna de sexo de los datos está normalizada: para hombre se establece en 1 y para mujer en 0. La columna de glucosa se establece en 1 si es > 120; De lo contrario, 0. En cuanto a la salida, si es positivo, se establece en 1 y si es negativo, en 0.

**El conjunto de datos tiene 9 columnas:**

Edad: Edad del paciente

Género: Sexo biológico del paciente (1 para masculino y 0 para femenino)

Frecuencia cardíaca: Número de latidos por minuto

Presión arterial sistólica: Presión arterial en las arterias cuando el corazón se contrae

Presión arterial diastólica: Presión arterial entre latidos

Glucemia: Nivel de glucosa en sangre del paciente

CK-mb: Enzima cardíaca liberada durante el daño del músculo cardíaco

Troponina: Biomarcador proteico altamente específico para la lesión del músculo cardíaco

Resultado: Etiqueta del resultado que indica si el paciente sufrió un infarto

Instituciones
Universidad de Kurdistán Hewler

Categorías
Medicina, Cardiopatía

DOI: Rashid, Tarik A.; Hassan, Bryar (2022), “Conjunto de datos de infarto”, Mendeley Data, V1, doi: 10.17632/wmhctcrt5v.1

**Preguntas de análisis SQL (Medicaldataset.csv)**

¿Cuál es la edad promedio de los pacientes?

¿Cuál es la proporción de pacientes hombres y mujeres?

¿Cuál es la frecuencia de resultados positivos vs negativos (infarto)?

¿Qué edad tienen en promedio los pacientes con resultado positivo?

¿Cuál es el ritmo cardíaco promedio por género?

¿Cuál es la presión sistólica y diastólica promedio para pacientes con resultado positivo?

¿Qué niveles promedio de CK-MB tienen los pacientes que no tuvieron un infarto?

¿Qué porcentaje de pacientes con troponina mayor a 0.4 resultaron positivos?

¿Cuál es la edad del paciente más joven y más viejo que tuvo un infarto?

¿Existe alguna diferencia significativa en los niveles de azúcar en sangre entre pacientes con y sin infarto?

In [1]:
import pandas as pd
import sqlite3

# Cargar el dataset
df = pd.read_csv("Medicaldataset.csv")

# Crear conexión a base de datos SQLite en memoria
conn = sqlite3.connect(":memory:")

# Cargar el DataFrame como tabla en SQLite
df.to_sql("medical", conn, index=False, if_exists="replace")

1319

In [2]:
#¿Cuál es la edad promedio de los pacientes?

query1 = "SELECT AVG(Age) AS Avg_Age FROM medical"
pd.read_sql_query(query1, conn)

,Avg_Age
0,56.191812


In [3]:
#¿Cuál es la proporción de pacientes hombres y mujeres?

query2 = """
SELECT
  CASE WHEN Gender = 1 THEN 'Male' ELSE 'Female' END AS Gender,
  COUNT(*) AS Total,
  ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM medical), 2) AS Percentage
FROM medical
GROUP BY Gender
"""
pd.read_sql_query(query2, conn)

,Gender,Total,Percentage
0,Female,449,34.04
1,Male,870,65.96


In [4]:
#¿Cuál es la frecuencia de resultados positivos vs negativos?

query3 = """
SELECT
  Result,
  COUNT(*) AS Count
FROM medical
GROUP BY Result
"""
pd.read_sql_query(query3, conn)


,Result,Count
0,negative,509
1,positive,810


In [5]:
#¿Qué edad tienen en promedio los pacientes con resultado positivo?

query4 = """
SELECT AVG(Age) AS Avg_Age_Positive
FROM medical
WHERE Result = 'positive'
"""
pd.read_sql_query(query4, conn)

,Avg_Age_Positive
0,58.766667


In [6]:
#¿Cuál es el ritmo cardíaco promedio por género?

query5 = """
SELECT
  CASE WHEN Gender = 1 THEN 'Male' ELSE 'Female' END AS Gender,
  AVG("Heart Rate") AS Avg_HeartRate
FROM medical
GROUP BY Gender
"""
pd.read_sql_query(query5, conn)


,Gender,Avg_HeartRate
0,Female,80.260579
1,Male,77.343678


In [7]:
#¿Cuál es la presión sistólica y diastólica promedio para pacientes con resultado positivo?

query6 = """
SELECT
  AVG("Systolic Blood Pressure") AS Avg_Systolic,
  AVG("Diastolic Blood Pressure") AS Avg_Diastolic
FROM medical
WHERE Result = 'positive'
"""
pd.read_sql_query(query6, conn)

,Avg_Systolic,Avg_Diastolic
0,126.739506,72.161728


In [8]:
#¿Qué niveles promedio de CK-MB tienen los pacientes que no tuvieron un infarto?

query7 = """
SELECT AVG("Ck-mb") AS Avg_Ckmb_Negative
FROM medical
WHERE Result = 'negative'
"""
pd.read_sql_query(query7, conn)

,Avg_Ckmb_Negative
0,2.555344


In [9]:
#¿Qué porcentaje de pacientes con troponina mayor a 0.4 resultaron positivos?

query8 = """
SELECT
  ROUND(
    (SELECT COUNT(*) FROM medical WHERE Troponin > 0.4 AND Result = 'positive') * 100.0 /
    (SELECT COUNT(*) FROM medical WHERE Troponin > 0.4),
    2
  ) AS Positive_Percentage
"""
pd.read_sql_query(query8, conn)

,Positive_Percentage
0,99.09


In [10]:
#¿Cuál es la edad del paciente más joven y más viejo que tuvo un infarto?

query9 = """
SELECT
  MIN(Age) AS Youngest_Positive,
  MAX(Age) AS Oldest_Positive
FROM medical
WHERE Result = 'positive'
"""
pd.read_sql_query(query9, conn)

,Youngest_Positive,Oldest_Positive
0,19,103


In [11]:
#¿Existe alguna diferencia significativa en los niveles de azúcar en sangre entre pacientes con y sin infarto?

query10 = """
SELECT
  Result,
  AVG("Blood Sugar") AS Avg_Blood_Sugar
FROM medical
GROUP BY Result
"""
pd.read_sql_query(query10, conn)

,Result,Avg_Blood_Sugar
0,negative,149.757760
1,positive,144.671605
